In [7]:
import tensorflow as tf
import librosa
#import tensorflow_datasets as tfds
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
#import cv2
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import os
import json

In [6]:
!pip install tqdm

  Using cached tqdm-4.66.5-py3-none-any.whl.metadata (57 kB)
Using cached tqdm-4.66.5-py3-none-any.whl (78 kB)


In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
tf.config.experimental.set_memory_growth(gpus[0], True)

In [3]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        # Set only the first GPU as visible
        tf.config.set_visible_devices(gpus[0], 'GPU')
        # Allow memory growth to allocate memory dynamically on the GPU
        tf.config.experimental.set_memory_growth(gpus[0], True)
        print("GPU configuration successful.")
    except RuntimeError as e:
        print(e)
else:
    print("No GPU detected.")

GPU configuration successful.


In [11]:
DATASET_PATH = "data/raw_data/"

kws_list=['on','off','up','down']
lang_index_dict={kws_list[i]:i for i in range(len(kws_list))}
print(lang_index_dict)

JSON_PATH = "data_new.json"
sr=16000
SAMPLES_TO_CONSIDER = 16000 # 1 sec. of audio

{'on': 0, 'off': 1, 'up': 2, 'down': 3}


In [5]:
# # Define Hamming window function
# def hamming_window(length):
#     return 0.54 - 0.46 * np.cos(2 * np.pi * np.arange(length) / (length - 1))

In [12]:
frame_length = int(sr * ( 32 / 1000))
hop_length = int(sr * ( 16 / 1000))

data = {
        "mapping": [],
        "labels": [],
        "MFCCs": [],
        "files": []
    }

j=0
# loop through all sub-dirs
for i, (dirpath, dirnames, filenames) in enumerate(os.walk(DATASET_PATH)):

    # ensure we're at sub-folder level
    if dirpath is not DATASET_PATH:

        # save label (i.e., sub-folder name) in the mapping
        label = dirpath.split("/")[-1]
        
        if label in kws_list:
            j = j+1
            print("\nProcessing: '{}'".format(label))
            # process all audio files in sub-dir and store MFCCs
            for f in filenames:
                file_path = os.path.join(dirpath, f)
                #print(file_path)
                # load audio file and slice it to ensure length consistency among different files
                signal,_=librosa.load(file_path,sr=sr)
                if len(signal)>=SAMPLES_TO_CONSIDER:
                    signal=signal[0:SAMPLES_TO_CONSIDER]
                else:
                    npad = SAMPLES_TO_CONSIDER - len(signal)
                    signal=np.pad(signal, pad_width=npad, mode='constant', constant_values=0)[npad:]

                signal=signal.astype(np.float32)
                
                f=librosa.feature.mfcc(y=signal, sr=sr, win_length = frame_length,hop_length=hop_length,n_mfcc=13, n_fft = frame_length,center=0)
                #print(f.shape)
                #print("{}- {}".format(">=" , f.shape))
                # store data for analysed track
                data["labels"].append(j-1)
                data["MFCCs"].append(f.T.tolist())
                data["files"].append(file_path)
                print("{}: {}- {} ".format(file_path, label , j-1))
                
    # save data in json file
    with open(JSON_PATH, "w") as fp:
        json.dump(data, fp, indent= 4)   #### indent= 4
                


Processing: 'down'
data/raw_data/down\00b01445_nohash_0.wav: down- 0 
data/raw_data/down\00b01445_nohash_1.wav: down- 0 
data/raw_data/down\00f0204f_nohash_0.wav: down- 0 
data/raw_data/down\0a2b400e_nohash_0.wav: down- 0 
data/raw_data/down\0a2b400e_nohash_1.wav: down- 0 
data/raw_data/down\0a2b400e_nohash_2.wav: down- 0 

Processing: 'off'
data/raw_data/off\0a196374_nohash_0.wav: off- 1 
data/raw_data/off\0a2b400e_nohash_2.wav: off- 1 
data/raw_data/off\0a2b400e_nohash_3.wav: off- 1 
data/raw_data/off\0a2b400e_nohash_4.wav: off- 1 
data/raw_data/off\0a5636ca_nohash_0.wav: off- 1 
data/raw_data/off\0a5636ca_nohash_1.wav: off- 1 

Processing: 'on'
data/raw_data/on\00b01445_nohash_0.wav: on- 2 
data/raw_data/on\0a2b400e_nohash_0.wav: on- 2 
data/raw_data/on\0a2b400e_nohash_1.wav: on- 2 
data/raw_data/on\0a7c2a8d_nohash_0.wav: on- 2 
data/raw_data/on\0a9f9af7_nohash_0.wav: on- 2 
data/raw_data/on\0a9f9af7_nohash_1.wav: on- 2 

Processing: 'up'
data/raw_data/up\00b01445_nohash_0.wav: up-